In [ ]:
# import tensorflow as tf

# a0 = tf.constant([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
# a1 = tf.constant([[11, 12], [13, 14], [15, 16], [17, 18], [19, 110]])
# a = tf.keras.layers.concatenate([a0, a1], axis=0)
# b = tf.constant([0, 3])
# print(tf.gather_nd(a, b))
# c = tf.Variable([3*2, 3*2 + 1])
# print(tf.gather(a, c, axis=0))
# print(a.shape)

In [1]:
import pickle
import numpy as np
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit import Chem
from IPython.display import SVG
from rdkit.Chem.Draw import MolToImage
from rdkit.Chem import Draw

def idxForSignificants(th_, aWeight):
    retlist = []
    for idx, ele in enumerate(aWeight):
        if ele >= th_:
            retlist.append(idx)
    return retlist

def sigIdxAtom(sigIdx, currSmiList):
    sigIdx_general, sigIdx_atom = [], []
    twoChars = {"AL":1, "al":1, "Al":1, "AU":1, "au":1, "Au":1, "MG":1, "mg":1, "Mg":1, "ZN":1, "zn":1, "Zn":1, "CA":1, "ca":1, "Ca":1, "NA":1, "na":1, "Na":1, "CL":1, "cl":1, "Cl":1, "FE":1, "fe":1, "Fe":1, "BR":1, "br":1, "Br":1}
    atomcnt = 0
    for idx, ele in enumerate(currSmiList):
        if ele in twoChars or ele.isalpha() or ele.isdigit():
            atomcnt += 1
            if idx in sigIdx:
                sigIdx_general.append(idx)
                sigIdx_atom.append(atomcnt)
    return sigIdx_general, sigIdx_atom

#due to the nature of our extracted substructure, we need to fill the hole
#if the atoms are 1 distance away (an atom is not included in the list) => fill the gap
#if the atoms are 2 distance away => discard the atom outlier
def idxMergeAtomLists(sigIdx_general, sigIdx_atom):
    if len(sigIdx_general) == 1:
        return sigIdx_general
    retlist = []
    twoChars = {"AL":1, "al":1, "Al":1, "AU":1, "au":1, "Au":1, "MG":1, "mg":1, "Mg":1, "ZN":1, "zn":1, "Zn":1, "CA":1, "ca":1, "Ca":1, "NA":1, "na":1, "Na":1, "CL":1, "cl":1, "Cl":1, "FE":1, "fe":1, "Fe":1, "BR":1, "br":1, "Br":1}
    for idx, ele in enumerate(sigIdx_general):
        if idx == len(sigIdx_general) - 1:
            break

        if sigIdx_atom[idx + 1] - sigIdx_atom[idx] > 2:
            continue
        else:#if sigIdx_atom[idx + 1] - sigIdx_atom[idx] == 1 or sigIdx_atom[idx + 1] - sigIdx_atom[idx] == 2:
            if ele not in retlist:#becuase we add the next element ...
                retlist.append(ele)
            tmplist = []
            for i in range(sigIdx_general[idx] + 1, sigIdx_general[idx + 1]):
                tmplist.append(i)
            retlist.extend(tmplist)
            retlist.append(sigIdx_general[idx + 1])
    return retlist

def atomListsIntoSmiFrags(mergedAtomList, currSmiList):
    retlist = []
    prevIndex = 0
    tmplist = []
    for i, ele in enumerate(mergedAtomList):
        if i != 0:
            if ele - prevIndex > 1:
                retlist.append("".join(tmplist))
                tmplist = []
        tmplist.append(currSmiList[ele])
        prevIndex = ele
        
    if tmplist != []:
        retlist.append("".join(tmplist))
    return retlist

"""
def idxSeparate_old(sigIdx, currSmiList):
    atomIdx, bondIdx = [], []
    atomCnt = 0
    bondCnt = 0
    mol = Chem.MolFromSmiles("".join(currSmiList))
    size = mol.GetNumAtoms()
    twoChars = {"AL":1, "al":1, "Al":1, "AU":1, "au":1, "Au":1, "MG":1, "mg":1, "Mg":1, "ZN":1, "zn":1, "Zn":1, "CA":1, "ca":1, "Ca":1, "NA":1, "na":1, "Na":1, "CL":1, "cl":1, "Cl":1, "FE":1, "fe":1, "Fe":1, "BR":1, "br":1, "Br":1}
    for idx, ele in enumerate(currSmiList):
        if atomCnt >= size:
            break
        if ele in twoChars or ele.isalpha():
            if idx in sigIdx:
                atomIdx.append(atomCnt)
            atomCnt += 1

    for start in atomIdx[:-1]:
        for end in atomIdx[::-1]:
            if start == end:
                break
            #print("".join(currSmiList), start, end)
            tmpbond = mol.GetBondBetweenAtoms(start, end)
            if tmpbond:
                bondIdx.append(start)
    return atomIdx, bondIdx
"""

#attnweights, teststr = pickle.load( open( "../save_weights", "rb" ) )
convweights, teststr = pickle.load( open( "save_weight1", "rb" ) )

In [ ]:
# conv_weights.shape == (batch_size, seq_len, self.d_model)
print(len(convweights))
layer1 = np.squeeze(convweights)#layer1 shape = (whole_size, seq_len, self.d_model)
layer1_max = np.amax(layer1, axis=2)#layer1T_sum shape = (whole_size, seq_len)
# print(layer1_sum[0], max(layer1_sum[0]))
allMols = []
allAtomIdx = []
allBondIdx = []
for i, ele in enumerate(layer1_max):
    th_Until7thW = sorted(ele)[-7]
    sigIdxList = idxForSignificants(th_Until7thW, ele)
    sigIdx_general, sigIdx_atom = sigIdxAtom(sigIdxList, teststr[i])
    sigIdx_atom_merged = idxMergeAtomLists(sigIdx_general, sigIdx_atom)
    smarts_list = atomListsIntoSmiFrags(sigIdx_atom_merged, teststr[i])
    hit_ats, hit_bonds = [], []
    for smart in smarts_list:
        patt = Chem.MolFromSmarts(smart)
        smi = "".join(teststr[i])
        #print(smi, sigIdxList, sigIdx_general, sigIdx_atom_merged)
        mol = Chem.MolFromSmiles(smi)
        hit_ats.extend(list(mol.GetSubstructMatch(patt)))
        for bond in patt.GetBonds():
            aid1 = hit_ats[bond.GetBeginAtomIdx()]
            aid2 = hit_ats[bond.GetEndAtomIdx()]
            hit_bonds.append(mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())
    allMols.append(mol)
    allAtomIdx.append(hit_ats)
    allBondIdx.append(hit_bonds)

# for i in range(10):
#     print(allAtomIdx[i], allBondIdx[i])
#img = MolToImage(mol, size=(600, 600), fitImage=False)#, highlightAtoms=atomIdx, highlightBonds=bondIdx)   
#img
#print(smi)
img = Draw.MolsToGridImage( allMols, molsPerRow=2, highlightAtomLists=allAtomIdx, highlightBondLists=allBondIdx, useSVG=True, subImgSize=(400, 400), maxMols=10)
img


In [2]:
bioalert_nr_ar_lbd = pickle.load( open( "nr-ar-lbd_alerts", "rb" ) )

In [3]:
dic = {}
act1, act0 = 0, 0
for idx, key in enumerate(bioalert_nr_ar_lbd['Substructure in Molecule']):#molecule duplicated
    actlabel = bioalert_nr_ar_lbd['Activity label'][idx]
    
    if actlabel == 1.0:
        act1 += 1
        dic[Chem.MolToSmiles(key)] = 1
    else:
        act0 += 1
        if Chem.MolToSmiles(key) in dic and dic[Chem.MolToSmiles(key)] == 1:
            dic[Chem.MolToSmiles(key)] = 1
        else:
            dic[Chem.MolToSmiles(key)] = 0
print(act1, act0)
print(len(dic))

cnt, cntPos = 0, 0
for currSmiList in teststr:
    currsmi = "".join(currSmiList)

    if currsmi in dic:
        cnt += 1
        if dic[currsmi] == 1:
            cntPos += 1
print(cnt, cntPos)

213 3159
1320
131 9


In [5]:
dic = {}
for idx, key in enumerate(bioalert_nr_ar_lbd['Substructure in Molecule']):#molecule duplicated
    smikey = Chem.MolToSmiles(key)
    if smikey in dic:
        dic[smikey].append(idx)
    else:
        dic[smikey] = [idx]
print(len(dic))

def howManyAshareB(A, B):
    share, notshare = 0, 0
    
    for ele in A:
        if ele in B:
            share += 1
        else:
            notshare += 1
    return share, notshare, share + notshare


layer1 = np.squeeze(convweights)#layer1 shape = (whole_size, seq_len, self.d_model)
layer1_max = np.amax(layer1, axis=2)#layer1T_sum shape = (whole_size, seq_len)
shared, notShared = 0, 0
for i, currSmiList in enumerate(teststr):
    currsmi = "".join(currSmiList)
    currmol = Chem.MolFromSmiles(currsmi)
    if currsmi in dic:
        th_Until7thW = sorted(layer1_max[i])[-7]
        sigIdxList = idxForSignificants(th_Until7thW, layer1_max[i])
        sigIdx_general, sigIdx_atom = sigIdxAtom(sigIdxList, teststr[i])
        sigIdx_atom_merged = idxMergeAtomLists(sigIdx_general, sigIdx_atom)
        smarts_list = atomListsIntoSmiFrags(sigIdx_atom_merged, teststr[i])
        
        isSmartsIn = False
        for idx in dic[currsmi]:
            sigAtomsInMol = list(currmol.GetSubstructMatch(bioalert_nr_ar_lbd['Substructure'][idx]))
            substructure_smi = Chem.MolToSmiles(bioalert_nr_ar_lbd['Substructure'][idx])
            #print(sigAtomsInMol, Chem.MolToSmiles(bioalert_nr_ar_lbd['Substructure'][idx]))
            for smart in smarts_list:
                if smart.lower() in substructure_smi.lower():
                    isSmartsIn = True
                    print(smart, substructure_smi)
        if isSmartsIn:
            shared += 1
        else:
            notShared += 1
print(shared, notShared)

1320
c(Cl)c ccc(Cl)c(c)C
O)CCl NC(=O)CCl
ccc ccc(C(=O)O)c(c)O
cc(-c c1ccc(-c2ccccc2)cc1
c2)c c1ccc(-c2ccccc2)cc1
c1ccccc CCOC(=O)c1ccccc1
COCC CCCCOCCOC
O C[C@@](C)(O)C(=O)CO
CCCCCC CCCCCC(=O)OCC
cc OCCN1CCNCC1
cc OCCN1CCNCC1
cc CCN1CCN(C)CC1
cc CCN1CCN(C)CC1
cc cc(c)C(c(c)c)N1CCN(CC)CC1
cc cc(c)C(c(c)c)N1CCN(CC)CC1
cc ccc(cc)C(c(cc)cc)N1CCN(C)CC1
cc ccc(cc)C(c(cc)cc)N1CCN(C)CC1
ccc ccc(cn)C(N)=O
ccc ccc(cn)C(N)=O
ccc NC(=O)c1cccnc1
ccc NC(=O)c1cccnc1
cc COc1ccccc1C
cc1 COc1ccccc1C
cc ccc(cc)NC(=O)c(c)c
cc Nc1ccc(C(=O)O)cc1
cc1 Nc1ccc(C(=O)O)cc1
cc(Cl)cc Cc1cc(Cl)ccc1O
c1cc cc1cccc(C)c1
Oc Oc1cccc2ccccc12
ccc2 Oc1cccc2ccccc12
C/C1 C=C(C)/C=C/C1=C(C)CCCC1(C)C
CC CCCCCOC(=O)CC
CCCCC CCCCCOC(=O)CC
CC COC(=O)CCCCC(=O)O
CCCCC COC(=O)CCCCC(=O)O
CCc cc(c)CCC
cc(Cl ccc(Cl)c(c)S
ccc ccc(cc)OS
OCC CCCOCC(C)O
OCC CCCOCC1CO1
1cc[n CCCCn1cc[n+](C)c1
cc cc(C)cc(c)O
cc ccc(cc)C(C)C
cc C=Cc(cc)cc
ccc(O Cc1ccc(O)cc1
ccc(O)c O=C(O)c1ccc(O)cc1
N1 CC[C@H](NC(=O)[C@@H]1CCCN1C(C)=O)C(N)=O
N1 CNC(=O)[C@@H]1C